# Index Management - Ungraph

Este notebook demuestra cómo gestionar índices en Neo4j para optimizar las búsquedas.

## Objetivos

1. **Crear índices vectoriales** - Para búsqueda por similitud semántica
2. **Crear índices full-text** - Para búsqueda por texto completo
3. **Crear índices regulares** - Para búsquedas por propiedades específicas
4. **Verificar índices existentes** - Listar y verificar índices
5. **Eliminar índices** - Gestión de índices
6. **Limpieza del grafo** - Eliminar datos y resetear

**Referencias:**
- [Neo4j Indexes](https://neo4j.com/docs/cypher-manual/current/indexes/)


In [ ]:
def add_src_to_path(path_folder: str):
    import sys
    from pathlib import Path
    base_path = Path().resolve()
    for parent in [base_path] + list(base_path.parents):
        candidate = parent / path_folder
        if candidate.exists():
            parent_dir = candidate.parent
            if str(parent_dir) not in sys.path:
                sys.path.insert(0, str(parent_dir))
            if str(candidate) not in sys.path:
                sys.path.append(str(candidate))
            return

add_src_to_path(path_folder="src")
add_src_to_path(path_folder="src/utils")

try:
    import ungraph
except ImportError:
    import src
    ungraph = src

from infrastructure.services.neo4j_index_service import Neo4jIndexService
from src.utils.graph_operations import graph_session

print(f"📦 Ungraph version: {ungraph.__version__}")


## Parte 1: Crear Índices

Creemos los diferentes tipos de índices necesarios para las búsquedas.


In [ ]:
# Crear servicio de índices
index_service = Neo4jIndexService()

# Crear todos los índices necesarios
print("🔧 Configurando todos los índices...\n")
index_service.setup_all_indexes()

print("✅ Índices configurados:")
print("   - Índice vectorial: chunk_embeddings (384 dimensiones)")
print("   - Índice full-text: chunk_content")
print("   - Índice regular: chunk_consecutive_idx")


## Parte 2: Verificar Índices Existentes

Listemos los índices que existen en la base de datos.


In [ ]:
# Listar índices existentes
driver = graph_session()
try:
    with driver.session() as session:
        # Índices regulares y full-text
        result = session.run("SHOW INDEXES")
        print("📊 Índices en la base de datos:\n")
        for record in result:
            print(f"  - {record.get('name', 'N/A')}: {record.get('type', 'N/A')}")
            print(f"    Estado: {record.get('state', 'N/A')}")
            print()
        
        # Índices vectoriales
        result = session.run("SHOW INDEXES YIELD name, type WHERE type = 'VECTOR'")
        vector_indexes = list(result)
        if vector_indexes:
            print("📊 Índices vectoriales:")
            for record in vector_indexes:
                print(f"  - {record['name']}")
finally:
    driver.close()


## Parte 3: Crear Índices Individuales

Podemos crear índices específicos según nuestras necesidades.


In [ ]:
# Crear índice vectorial personalizado
print("🔧 Creando índice vectorial personalizado...")
index_service.setup_vector_index(
    index_name="custom_embeddings",
    node_label="Chunk",
    property_name="embeddings",
    dimensions=384
)

# Crear índice full-text con analyzer específico
print("🔧 Creando índice full-text con analyzer inglés...")
index_service.setup_fulltext_index(
    index_name="chunk_content_en",
    node_label="Chunk",
    property_name="page_content",
    analyzer="english"
)

print("✅ Índices personalizados creados")


## Parte 4: Eliminar Índices

Podemos eliminar índices que ya no necesitamos.


In [ ]:
# Eliminar un índice específico
index_to_drop = "custom_embeddings"  # Cambiar según necesidad

print(f"🗑️  Eliminando índice: {index_to_drop}")
try:
    index_service.drop_index(index_to_drop)
    print(f"✅ Índice '{index_to_drop}' eliminado")
except Exception as e:
    print(f"⚠️  Error eliminando índice: {e}")


## Parte 5: Limpieza del Grafo

Podemos limpiar el grafo eliminando nodos específicos o todo el contenido.


In [ ]:
# Limpiar nodos específicos (ejemplo: solo Chunks)
print("⚠️  ADVERTENCIA: Esto eliminará todos los Chunks del grafo")
print("   Descomenta la siguiente línea para ejecutar:\n")

# index_service.clean_graph(node_labels=["Chunk"])

# Para limpiar todo el grafo:
# index_service.clean_graph()  # Elimina todos los nodos

print("💡 Usa clean_graph() con cuidado. Asegúrate de tener backups.")


## Resumen y Mejores Prácticas

### Tipos de Índices

1. **Vectorial**: Para búsqueda por similitud semántica (embeddings)
2. **Full-text**: Para búsqueda por texto completo (page_content)
3. **Regular**: Para búsquedas rápidas por propiedades específicas

### Mejores Prácticas

1. **Crear índices antes de ingerir**: Mejora el rendimiento de la ingesta
2. **Usar setup_all_indexes()**: Crea todos los índices necesarios de una vez
3. **Verificar índices**: Usa SHOW INDEXES para verificar el estado
4. **Eliminar índices no usados**: Libera espacio y mejora rendimiento
5. **Limpieza cuidadosa**: clean_graph() elimina datos permanentemente

## Referencias

- [Neo4j Indexes](https://neo4j.com/docs/cypher-manual/current/indexes/)
- [Vector Indexes](https://neo4j.com/docs/cypher-manual/current/indexes-for-vector-search/)
